---
title: IDs from ARTHEMIS
format:
  html:
    code-fold: true
output-file: artemis.html
---

## Background

ARTEMIS spacecrafts will be exposed in the solar wind at 1 AU during its orbits around the Moon. So it's very interesting to look into its data.

- For time inteval for THEMIS-B in solar wind, see [Link](https://omniweb.gsfc.nasa.gov/ftpbrowser/themis_b_sw.txt)
- For time inteval for THEMIS-C in solar wind, see [Link](https://omniweb.gsfc.nasa.gov/ftpbrowser/themis_c_sw.txt)


## Setup

Need to run command in shell first as `pipeline` is project-specific command

```{sh}
kedro pipeline create themis
```

To get candidates data, run `kedro run --from-inputs=jno.feature_1s --to-outputs=candidates.jno_1s`

In [ ]:
#| hide
#| default_exp pipelines/themis/pipeline
%load_ext autoreload
%autoreload 2
from nbdev.showdoc import *


In [ ]:
# | code-summary: import all the packages needed for the project
# | output: hide
# | export

from ids_finder.core import *
from ids_finder.utils.basic import DataConfig
from fastcore.utils import *
from fastcore.test import *

import polars as pl
import pandas
import numpy as np
import xarray as xr


from datetime import timedelta, datetime
from loguru import logger

#### `Kerdo`

In [ ]:
# | export
from kedro.pipeline import Pipeline, node
from kedro.pipeline.modular_pipeline import pipeline

In [ ]:
# | eval: false
from ids_finder.utils.basic import load_catalog

In [ ]:
catalog = load_catalog()

jno_start_date = catalog.load("params:jno_start_date")
jno_end_date = catalog.load("params:jno_end_date")
trange = [jno_start_date, jno_end_date]

## Magnetic field data pipeline

- For convenience, we choose magnetic field data in **GSE** coordinate system
- The `fgs` data are in 3-4s resolution

### Downloading data

In [ ]:
# | export
import speasy as spz
from speasy import SpeasyVariable

In [ ]:
# | export
def check_dataype(ts: int):
    ts = pandas.Timedelta(seconds=ts)
    fgs_ts = timedelta(seconds=3)
    fgl_ts = timedelta(seconds=0.1)

    if ts > fgs_ts:
        datatype = "fgs"
    elif ts > fgl_ts:
        datatype = "fgl"
    else:
        datatype = "fgh"
    return datatype

In [ ]:
# | export
def download_mag_data(
    trange, probe: str = "b", datatype="fgs", coord="gse"
) -> SpeasyVariable:
    match probe:
        case "b":
            sat = "thb"

    product = f"cda/{sat.upper()}_L2_FGM/{sat}_{datatype}_{coord}"
    data = spz.get_data(product, trange, disable_proxy=True)

    return data


def spz2df(raw_data: SpeasyVariable):
    return pl.from_dataframe(raw_data.to_dataframe().reset_index()).rename(
        {"index": "time"}
    )


def load_mag_data(
    start: str,
    end: str,
    ts: int = None,  # time resolution
    probe: str = "b",
    coord="gse",
):
    trange = [start, end]

    datatype = check_dataype(ts)

    data = download_mag_data(trange, probe, datatype, coord)
    return spz2df(data).lazy()

### Preprocessing data

In [ ]:
# | export
from ids_finder.utils.basic import resample

In [ ]:
# | export
def preprocess_mag_data(
    raw_data: pl.LazyFrame,
    datatype: str = None,
    coord: str = "gse",
) -> pl.LazyFrame:
    """
    Preprocess the raw dataset (only minor transformations)

    - Applying naming conventions for columns
    - Dropping duplicate time
    - Changing storing format to `parquet`

    """

    datatype = datatype.upper()
    name_mapping = {
        f"Bx {datatype}-D": "B_x",
        f"By {datatype}-D": "B_y",
        f"Bz {datatype}-D": "B_z",
    }

    return raw_data.sort("time").unique("time").rename(name_mapping)

### Processing data

In [ ]:
# | export
from ids_finder.utils.basic import partition_data_by_year

In [ ]:
# | export
def process_mag_data(
    raw_data: pl.LazyFrame,
    ts: int = None,  # time resolution
) -> pl.DataFrame | Dict[str, pl.DataFrame]:
    """
    Partitioning data, for the sake of memory
    """

    every = timedelta(seconds=ts)
    period = 2 * every

    return raw_data.pipe(resample, every=every, period=period).pipe(
        partition_data_by_year
    )

### Pipeline

In [ ]:
# | exports
def create_mag_data_pipeline(
    sat_id: str,  # satellite id, used for namespace
    ts: int = 1,  # time resolution, in seconds
    tau: str = "60s",  # time window
    **kwargs,
) -> Pipeline:
    
    datatype = check_dataype(
        ts
    )  # get the datatype from the time resolution, which is reasonable but not always is the case
    ts_str = f"ts_{ts}s"

    node_load_data = node(
        load_mag_data,
        inputs=dict(
            start="params:start_date",
            end="params:end_date",
            ts="params:mag.time_resolution",
        ),
        outputs=f"raw_mag",
        name=f"load_{sat_id.upper()}_magnetic_field_data",
    )

    node_preprocess_data = node(
        preprocess_mag_data,
        inputs=dict(
            raw_data=f"raw_mag",
            datatype=datatype,
        ),
        outputs=f"inter_mag_{datatype}",
        name=f"preprocess_{sat_id.upper()}_magnetic_field_data",
    )

    node_process_data = node(
        process_mag_data,
        inputs=dict(
            raw_data=f"inter_mag_{datatype}",
            ts="params:mag.time_resolution",
        ),
        outputs=f"primary_mag_{ts_str}",
        name=f"process_{sat_id.upper()}_magnetic_field_data",
    )

    node_extract_features = node(
        extract_features,
        inputs=[f"primary_mag_{ts_str}", "params:tau", "params:mag"],
        outputs=f"feature_{ts_str}_tau_{tau}",
        name=f"extract_{sat_id}_features",
    )

    nodes = [
        node_load_data,
        node_preprocess_data,
        node_process_data,
        node_extract_features,
    ]

    pipelines = pipeline(
        nodes,
        namespace=sat_id,
        parameters={
            "params:start_date": "params:jno_start_date",
            "params:end_date": "params:jno_end_date",
            "params:tau": "params:tau",
        },
    )

    return pipelines

##### Test

In [ ]:
# | eval: false
ts = "4s"
thb_inter_mag = catalog.load(f"thb.inter_mag_{ts}")
thb_inter_mag.columns

In [ ]:
from ids_finder.utils.basic import check_fgm

## State data pipeline

We use low resolution [OMNI data](https://omniweb.gsfc.nasa.gov/ow.html) for plasma state data, see [details](https://spdf.gsfc.nasa.gov/pub/data/omni/low_res_omni/omni2.text).

- Data gaps were filled with dummy numbers for the missing hours or entire
  days to make all files of equal length.  The character '9' is used to
  fill all fields for missing data according to their format, e.g.
  ' 9999.9' for a field with the FORTRAN format F7.1. Note that format F7.1
  below really means (1X,F6.1),etc.

```
The flow OMNI "phi" angle is opposite GSE "phi" angle, threrfore, Flow-vector cartesian components in GSE coordinates may be derived from the given speed and angles as

Vx = - V * cos(theta) * cos(phi)
Vy = + V * cos(theta) * sin(phi)
Vz = + V * sin(theta)
and vise versa: two angles may be derived from the given speed and Vx,Vy,Vz comp. as  
          a_theta=vz/V
          theta=(180.*asin(a_theta))/!PI
         a_phi=Vy/(-Vx)
        phi=(180.*atan(a_phi))/!PI
```

```
   (*)   Quasi-GSE for the flow longitude angle means the angle increases from zero
         to positive values as the flow changes from being aligned along the -X(GSE)
         axis towards the +Y(GSE) axis.  The flow longitude angle is positive for 
         flow from west of the sun, towards +Y(GSE).
         The flow latitude angle is positive for flow from south of the sun, 
         towards +Z(GSE)
``````                  

### Downloading data

In [ ]:
# | export
from ids_finder.utils.basic import cdf2pl, pmap

In [ ]:
# | export


def download_state_data(
    start: str = None,
    end: str = None,
    ts: str = None,  # time resolution
    probe: str = None,
    coord: str = None,
):
    import pyspedas

    trange = [start, end]
    files = pyspedas.omni.data(trange=trange, datatype="hour", downloadonly=True)
    return files


def load_state_data(
    start: str = None,
    end: str = None,
    ts: str = None,  # time resolution
    vars: dict = None,
):
    files = download_state_data(start, end, ts, vars)
    df: pl.LazyFrame = pl.concat(files | pmap(cdf2pl, var_names=list(vars)))
    return df

### Preprocessing data

In [ ]:
# | export
def preprocess_state_data(
    raw_data: pl.LazyFrame,
    vars: dict,
) -> pl.LazyFrame:
    """
    Preprocess the raw dataset (only minor transformations)

    - Applying naming conventions for columns
    - Extracting variables from `CDF` files, and convert them to DataFrame
    """

    columns_name_mapping = {key: value["COLNAME"] for key, value in vars.items()}

    return raw_data.rename(columns_name_mapping)

Also we have additional data file that indicate if `THEMIS` is in solar wind or not.

In [ ]:
# | export
def preprocess_sw_state_data(
    raw_data: pandas.DataFrame,
) -> pl.LazyFrame:
    """
    - Applying naming conventions for columns
    - Parsing and typing data (like from string to datetime for time columns)
    """

    return pl.from_dataframe(raw_data).with_columns(
        # Note: For `polars`, please either specify both hour and minute, or neither.
        pl.concat_str(pl.col("start"), pl.lit(" 00")).str.to_datetime(
            format="%Y %j %H %M"
        ),
        pl.concat_str(pl.col("end"), pl.lit(" 00")).str.to_datetime(
            format="%Y %j %H %M"
        ),
    )

### Processing data

In [ ]:
# | export
def flow2gse(df: pl.LazyFrame) -> pl.LazyFrame:
    """
    - Transforming solar wind data from `Quasi-GSE` coordinate to GSE coordinate system
    """
    plasma_speed = pl.col("plasma_speed")
    sw_theta = pl.col("sw_vel_theta")
    sw_phi = pl.col("sw_vel_phi")

    return df.with_columns(
        sw_vel_gse_x=-plasma_speed * sw_theta.cos() * sw_phi.cos(),
        sw_vel_gse_y=+plasma_speed * sw_theta.cos() * sw_phi.sin(),
        sw_vel_gse_z=+plasma_speed * sw_theta.sin(),
    ).drop(["sw_theta", "sw_phi"])


def filter_tranges(time: pl.Series, tranges: Tuple[list, list]):
    """
    - Filter data by time ranges, return the indices of the time that are in the time ranges
    """

    starts = tranges[0]
    ends = tranges[1]

    start_indices = time.search_sorted(starts)
    end_indices = time.search_sorted(ends)

    return np.concatenate(
        [
            np.arange(start_index, end_index + 1)
            for start_index, end_index in zip(start_indices, end_indices)
        ]
    )


def add_state(l_df: pl.LazyFrame, l_state: pl.LazyFrame):
    state = l_state.collect()
    df = l_df.collect()

    start = state.get_column("start")
    end = state.get_column("end")

    time = df.get_column("time")

    indices = filter_tranges(time, (start, end))

    return (
        df.with_row_count()
        .with_columns(
            state=pl.when(pl.col("row_nr").is_in(indices)).then(1).otherwise(0)
        )
        .drop("row_nr")
    )


def process_state_data(df: pl.LazyFrame, state: pl.LazyFrame = None) -> pl.LazyFrame:
    """
    - Transforming data to GSE coordinate system
    - Combine state data with additional plasma state data
    """

    return (
        df.pipe(flow2gse)
        .pipe(add_state, state)
        .rename(
            {
                "sw_vel_gse_x": "v_x",
                "sw_vel_gse_y": "v_y",
                "sw_vel_gse_z": "v_z",
            }
        )
    )

### Pipelines

In [ ]:
# | export
def create_state_data_pipeline(
    sat_id,
    ts: str = "1h",  # time resolution
) -> Pipeline:
    node_load_data = node(
        load_state_data,
        inputs=dict(
            start="params:start_date",
            end="params:end_date",
            vars="params:omni_vars",
        ),
        outputs=f"raw_state",
        name=f"load_{sat_id.upper()}_state_data",
    )

    node_preprocess_data = node(
        preprocess_state_data,
        inputs=dict(
            raw_data=f"raw_state",
            vars="params:omni_vars",
        ),
        outputs=f"inter_state_{ts}",
        name=f"preprocess_{sat_id.upper()}_state_data",
    )

    node_preprocess_sw_state = node(
        preprocess_sw_state_data,
        inputs=f"raw_state_sw",
        outputs=f"inter_state_sw",
        name=f"preprocess_{sat_id.upper()}_solar_wind_state_data",
    )

    node_process_data = node(
        process_state_data,
        inputs=[f"inter_state_{ts}", f"inter_state_sw"],
        outputs=f"primary_state_{ts}",
        name=f"process_{sat_id.upper()}_state_data",
    )

    nodes = [
        node_load_data,
        node_preprocess_data,
        node_preprocess_sw_state,
        node_process_data,
    ]
    pipelines = pipeline(
        nodes,
        namespace=sat_id,
        parameters={
            "params:omni_vars": "params:omni_vars",
            "params:start_date": "params:jno_start_date",
            "params:end_date": "params:jno_end_date",
        },
    )

    return pipelines

In [ ]:
# | eval: false
catalog.load("thb.primary_state_1h").collect().describe()
# catalog.load('thb.feature_tau_60s').collect()

## Processing the whole data

In [ ]:
# | export
from ids_finder.pipelines.default import create_candidate_pipeline

In [ ]:
#| export
from ids_finder.utils.basic import filter_tranges_df

def filter_sw_events(events: pl.LazyFrame, sw_state: pl.LazyFrame) -> pl.LazyFrame:
    
    start, end = sw_state.select(['start', 'end']).collect()
    sw_events = filter_tranges_df(events.collect(), (start, end))
    
    return sw_events

def create_sw_events_pipeline(
    sat_id,
    tau: int = 60,
    ts_mag: int = 1,
    
):
  
    ts_mag_str = f"ts_{ts_mag}s"
    tau_str = f"tau_{tau}s"
    
    node_filter_sw_events = node(
        filter_sw_events,
        inputs=[
            f"candidates.{sat_id}_{ts_mag_str}_{tau_str}",
            f"{sat_id}.inter_state_sw",
        ],
        outputs=f"events.sw.{sat_id}_{ts_mag_str}_{tau_str}"
        
    )

    nodes = [node_filter_sw_events]
    return pipeline(nodes)

In [ ]:
# | export
def create_pipeline(
    sat_id="thb",
    tau=60, # time window, in seconds
    ts_state="1h",  # time resolution of state data
    ts_mag = 1, # time resolution of mag data, in seconds
) -> Pipeline:
    return (
        create_mag_data_pipeline(sat_id, tau=tau)
        + create_state_data_pipeline(sat_id, ts=ts_state)
        + create_candidate_pipeline(sat_id, tau=tau, ts_mag= ts_mag, ts_state=ts_state)
        + create_sw_events_pipeline(sat_id, tau=tau, ts_mag= ts_mag)
    )

In [ ]:
# | eval: false
# catalog.load('thb.inter_mag_4s').collect().describe()
# catalog.load('thb.primary_state_1h').collect().describe()

## Obsolete codes

### Check and preprocess the data

As we are only interested in the data when THEMIS is in the solar wind, for simplicity we will only keep the data when `X, SSE` and `X, GSE` is positive.

- State data time resolution is 1 minute...

- FGS data time resolution is 4 second...

In [ ]:
def get_thm_state(sat):
    sat_pos_sse_files = f"../data/{sat}_pos_sse.parquet"
    sat_pos_sse = pl.scan_parquet(sat_pos_sse_files).set_sorted("time")
    sat_pos_gse_files = f"../data/{sat}_pos_gse.parquet"
    sat_pos_gse = pl.scan_parquet(sat_pos_gse_files).set_sorted("time")
    sat_state = sat_pos_sse.join(sat_pos_gse, on="time", how="inner")
    return sat_state

In [ ]:
%%markdown
df = (
    sat_state_sw.upsample("time", every="1m")
    .group_by_dynamic("time", every="1d")
    .agg(pl.col("X, SSE").null_count().alias("null_count"))
    .with_columns(
        pl.when(pl.col("null_count") > 720).then(0).otherwise(1).alias("availablity")
    )
)

properties = {
    'width': 800,
}

chart1 = alt.Chart(df).mark_point().encode(
    x='time',
    y='null_count'
).properties(**properties)

chart2  = alt.Chart(df).mark_point().encode(
    x='time',
    y='availablity'
).properties(**properties)

(chart1 & chart2)